<a href="https://colab.research.google.com/github/rabbitxyt/stock/blob/main/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# 【1】安装库和检测环境

try:
    import google.auth
    import gspread
    import pandas
    import requests
    import openpyxl
    import pytz
    from googleapiclient.discovery import build
    from gspread_formatting import *
except ImportError:
    !pip install google-auth gspread gspread-formatting google-api-python-client pandas requests openpyxl pytz

In [ ]:
# 检测是否在Google Colab环境中运行
try:
    from google.colab import drive, auth  # 用于在Google Colab中挂载Google Drive和进行用户认证
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # 在Google Colab中运行的设置
    drive.mount('/content/drive') # 挂载Google Drive
    auth.authenticate_user() # 认证和授权
    from google.auth import default
    creds, _ = default()
else:
    # 在本地运行的设置，使用服务账号凭证文件
    from google.oauth2.service_account import Credentials
    creds = Credentials.from_service_account_file('path/to/your/service_account.json')  # 请替换为你的服务账号文件路径

# 使用凭证登录
gc = gspread.authorize(creds)

In [3]:
# 【2】数据获取

# 2.1 输入参数

# 获取当前日期，并将其格式化为"YYYY-MM-DD"的字符串
today_date = datetime.now().strftime("%Y-%m-%d")

# 设置股票代码和API Key
stock_code = "AAPL"
api_key = "VAVS6TS3HVTPGWQ9"



In [4]:
# 2.2 从API获取股票数据并转换为Pandas DataFrame，存放在 original_data 表中

# 获取股票数据的函数
def get_stock_data(stock_code, api_key):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_WEEKLY",
        "symbol": stock_code,
        "apikey": api_key
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    return data

# 获取股票数据
stock_data = get_stock_data(stock_code, api_key)

# 将数据转换为Pandas DataFrame
time_series = stock_data.get("Weekly Time Series", {})
data = {
    "Date": [],
    "Close": [],
    "Volume": []
}

for date, metrics in time_series.items():
    data["Date"].append(date)
    data["Close"].append(metrics["4. close"])
    data["Volume"].append(metrics["5. volume"])

original_data = pd.DataFrame(data)

In [5]:
# 【3】处理数据

# 3.1 只获取截止到一周收盘的数据

from datetime import datetime, timedelta
import pytz


# 将 Date 列转换为 datetime 类型
original_data['Date'] = pd.to_datetime(original_data['Date'])

# 将 Volume 列转换为数值类型
original_data['Volume'] = pd.to_numeric(original_data['Volume'], errors='coerce')


# 获取当前时间并指定时区，例如使用美国东部时间（ET）
now = datetime.now(pytz.timezone('US/Eastern'))

# 判断今天是星期几
weekday = now.weekday()

# 判断是否已经过了周五的市场收盘时间（下午4点）
friday_close_time = now.replace(hour=17, minute=0, second=0, microsecond=0)

# 如果今天是周六(5)或周日(6)或者是周五且已经过了收盘时间
if weekday > 5 or (weekday == 5 and now > friday_close_time):
    # 获取本周日的日期
    end_date = now + timedelta(days=(6 - weekday))
else:
    # 获取上周日的日期
    end_date = now - timedelta(days=(weekday + 1))

# 将 end_date 转换为不含时区信息的 datetime 对象
end_date = end_date.replace(tzinfo=None)

# 过滤只保留一周收盘后的数据
original_data = original_data[original_data['Date'] <= end_date].reset_index(drop=True)

In [6]:
# 【3】数据分析

# 3.1 只获取截止到一周收盘的数据

from datetime import datetime, timedelta

# 将 Date 列转换为 datetime 类型
original_data['Date'] = pd.to_datetime(original_data['Date'])

# 将 Volume 列转换为数值类型
original_data['Volume'] = pd.to_numeric(original_data['Volume'], errors='coerce')


# 获取当前时间并指定时区，例如使用美国东部时间（ET）
now = datetime.now(pytz.timezone('US/Eastern'))

# 判断今天是星期几
weekday = now.weekday()

# 判断是否已经过了周五的市场收盘时间（下午4点）
friday_close_time = now.replace(hour=17, minute=0, second=0, microsecond=0)

# 如果今天是周六(5)或周日(6)或者是周五且已经过了收盘时间
if weekday > 5 or (weekday == 5 and now > friday_close_time):
    # 获取本周日的日期
    end_date = now + timedelta(days=(6 - weekday))
else:
    # 获取上周日的日期
    end_date = now - timedelta(days=(weekday + 1))

# 将 end_date 转换为不含时区信息的 datetime 对象
end_date = end_date.replace(tzinfo=None)

# 过滤只保留一周收盘后的数据
original_data = original_data[original_data['Date'] <= end_date].reset_index(drop=True)

In [7]:
# 3.2 添加 Close_Change 列，表示本周收盘价比上周高了还是低了

original_data['Close_Change'] = 'NA'  # 默认值
for i in range(len(original_data) - 1):
    if original_data.loc[i, 'Close'] > original_data.loc[i + 1, 'Close']:
        original_data.loc[i, 'Close_Change'] = 'Up'
    elif original_data.loc[i, 'Close'] < original_data.loc[i + 1, 'Close']:
        original_data.loc[i, 'Close_Change'] = 'Down'

In [8]:
# 3.3 添加 HighOrLow 列，表示这个价格是不是一个高点/低点

original_data['HighOrLow'] = 'NA'  # 默认值

# 特殊处理第一行
if original_data.loc[0, 'Close_Change'] == 'Up':
    original_data.loc[0, 'HighOrLow'] = 'High'
elif original_data.loc[0, 'Close_Change'] == 'Down':
    original_data.loc[0, 'HighOrLow'] = 'Low'

# 处理其他行
for i in range(1, len(original_data) - 1):
    current_close = original_data.loc[i, 'Close']
    previous_close = original_data.loc[i - 1, 'Close']
    next_close = original_data.loc[i + 1, 'Close']

    if current_close > previous_close and current_close > next_close:
        original_data.loc[i, 'HighOrLow'] = 'High'
    elif current_close < previous_close and current_close < next_close:
        original_data.loc[i, 'HighOrLow'] = 'Low'
    else:
        original_data.loc[i, 'HighOrLow'] = ''

In [9]:
# 3.4 根据收盘价高低和成交量计算 OBV 列

row_count = len(original_data)

# 添加 OBV 列并初始化为 0
original_data['OBV'] = 0

# 从倒数第二行开始向上遍历，计算 OBV 列的值
for i in range(len(original_data) - 2, -1, -1):
    if original_data.loc[i, 'Close_Change'] == 'Down':
        original_data.loc[i, 'OBV'] = original_data.loc[i + 1, 'OBV'] - original_data.loc[i, 'Volume']
    elif original_data.loc[i, 'Close_Change'] == 'Up':
        original_data.loc[i, 'OBV'] = original_data.loc[i + 1, 'OBV'] + original_data.loc[i, 'Volume']

# 将 OBV 列移动到 Close 列之后，Volume 列之前
obv_column = original_data.pop('OBV')
original_data.insert(original_data.columns.get_loc('Volume'), 'OBV', obv_column)

In [10]:
# 3.5 寻找背离的数据点

#初始化新列
original_data['Index'] = 0
original_data['Found_Row'] = pd.NA
original_data['Found_Date'] = pd.NaT
original_data['Found_Close'] = pd.NA
original_data['Found_OBV'] = pd.NA

# 遍历 original_data 中的每一行
for i in range(len(original_data)):
    if original_data.loc[i, 'HighOrLow'] == 'High':
        for j in range(i + 1, len(original_data)):
            if original_data.loc[j, 'HighOrLow'] == 'High':
                # 返回找到行的信息
                original_data.loc[i, 'Found_Row'] = j
                original_data.loc[i, 'Found_Date'] = original_data.loc[j, 'Date']
                original_data.loc[i, 'Found_Close'] = original_data.loc[j, 'Close']
                original_data.loc[i, 'Found_OBV'] = original_data.loc[j, 'OBV']
                # 判断 Index 值
                if original_data.loc[i, 'Close'] > original_data.loc[j, 'Close'] and original_data.loc[i, 'OBV'] < original_data.loc[j, 'OBV']:
                    original_data.loc[i, 'Index'] = 1
                elif original_data.loc[i, 'Close'] < original_data.loc[j, 'Close'] and original_data.loc[i, 'OBV'] > original_data.loc[j, 'OBV']:
                    original_data.loc[i, 'Index'] = 2
                break
    elif original_data.loc[i, 'HighOrLow'] == 'Low':
        for j in range(i + 1, len(original_data)):
            if original_data.loc[j, 'HighOrLow'] == 'Low':
                # 返回找到行的信息
                original_data.loc[i, 'Found_Row'] = j
                original_data.loc[i, 'Found_Date'] = original_data.loc[j, 'Date']
                original_data.loc[i, 'Found_Close'] = original_data.loc[j, 'Close']
                original_data.loc[i, 'Found_OBV'] = original_data.loc[j, 'OBV']
                # 判断 Index 值
                if original_data.loc[i, 'Close'] < original_data.loc[j, 'Close'] and original_data.loc[i, 'OBV'] > original_data.loc[j, 'OBV']:
                    original_data.loc[i, 'Index'] = 3
                elif original_data.loc[i, 'Close'] > original_data.loc[j, 'Close'] and original_data.loc[i, 'OBV'] < original_data.loc[j, 'OBV']:
                    original_data.loc[i, 'Index'] = 4
                break


In [11]:
# 【4】创建Google Sheets文件

file_name = f"{stock_code}"

# 检查Google Drive中是否存在同名文件
try:
    spreadsheet = gc.open(file_name)
    print(f"Spreadsheet '{file_name}' already exists.")
except gspread.exceptions.SpreadsheetNotFound:
    print(f"Creating new file '{file_name}'.")
    spreadsheet = gc.create(file_name)
    worksheet = spreadsheet.add_worksheet(title="Parameter", rows="100", cols="20")

    # 删除默认的 Sheet1
    if 'Sheet1' in [ws.title for ws in spreadsheet.worksheets()]:
        spreadsheet.del_worksheet(spreadsheet.worksheet('Sheet1'))

Spreadsheet 'AAPL' already exists.


In [ ]:
# 4.2 处理参数工作表

try:
    # 打开并检查 "Parameter" 表中的日期
    worksheet = spreadsheet.worksheet("Parameter")
    existing_date = worksheet.acell('B1').value
    print(f"Existing file '{file_name}' has a 'Parameter' sheet with Date: {existing_date}")

    overwrite = input(f"Do you want to overwrite the existing 'Parameter' sheet? (yes/no): ").strip().lower()
    if overwrite != 'yes':
        print("Operation aborted by the user.")
        exit()
    else:
        print(f"Overwriting existing 'Parameter' sheet in '{file_name}'.")
        worksheet.clear()  # 清空现有的表内容
except gspread.exceptions.WorksheetNotFound:
    print("Worksheet 'Parameter' not found, creating new 'Parameter' sheet.")
    worksheet = spreadsheet.add_worksheet(title="Parameter", rows="100", cols="20")

# 在工作表中写入参数
worksheet.update_acell('A1', "Last Create Date")
worksheet.update_acell('B1', today_date)
worksheet.update_acell('A2', "Stock Code")
worksheet.update_acell('B2', stock_code)
worksheet.update_acell('A3', "API Key")
worksheet.update_acell('B3', api_key)

In [ ]:
# 4.3 处理历史背离数据工作表

styled_data_sheet_name = "历史背离数据"
try:
    styled_data_sheet = spreadsheet.worksheet(styled_data_sheet_name)
    styled_data_sheet.clear()  # 清空现有的工作表内容
except gspread.exceptions.WorksheetNotFound:
    styled_data_sheet = spreadsheet.add_worksheet(title=styled_data_sheet_name, rows="1000", cols="20")

# 过滤数据
filtered_data = original_data[original_data['HighOrLow'] != '']
filtered_data = filtered_data[filtered_data['Index'].isin([1, 2, 3])]

# 选择需要显示的列
selected_columns = ["Found_Date", "Found_Close", "Found_OBV", "Date", "Close", "OBV", "Index"]
filtered_data = filtered_data[selected_columns]

# 确保 'Date' 和 'Found_date' 列的类型是 datetime，如果不是需要先转换
filtered_data[['Date', 'Found_Date']] = filtered_data[['Date', 'Found_Date']].apply(pd.to_datetime)

# 将 'Date' 和 'Found_date' 列转换为仅包含日期的格式并转换为字符串
filtered_data[['Date', 'Found_Date']] = filtered_data[['Date', 'Found_Date']].apply(lambda x: x.dt.strftime('%Y-%m-%d'))

# 将 'Close' 和 'Found_Close' 列转换为数值类型
filtered_data['Close'] = pd.to_numeric(filtered_data['Close'], errors='coerce')
filtered_data['Found_Close'] = pd.to_numeric(filtered_data['Found_Close'], errors='coerce')

# 将 'Close' 和 'Found_Close' 列格式化为小数点后两位并转换为字符串
filtered_data['Close'] = filtered_data['Close'].map('{:.2f}'.format)
filtered_data['Found_Close'] = filtered_data['Found_Close'].map('{:.2f}'.format)

# 确保 'OBV' 和 'Found_OBV' 列转换为数值类型
filtered_data['OBV'] = pd.to_numeric(filtered_data['OBV'], errors='coerce')
filtered_data['Found_OBV'] = pd.to_numeric(filtered_data['Found_OBV'], errors='coerce')

# 缩小 'OBV' 和 'Found_OBV' 列的值，并取整
scale_factor = 1e7  # 设置缩放因子
filtered_data['OBV'] = (filtered_data['OBV'] / scale_factor).round().astype(int)
filtered_data['Found_OBV'] = (filtered_data['Found_OBV'] / scale_factor).round().astype(int)

# 将 DataFrame 转换为列表格式
data_to_write = [filtered_data.columns.values.tolist()] + filtered_data.values.tolist()

# 将数据插入 Google Sheets
styled_data_sheet.update(data_to_write)

# 使用 gspread-formatting 设置高亮颜色
from gspread_formatting import CellFormat, Color, format_cell_range

# 定义颜色
def get_color(index):
    if index == 3:
        return Color(0.56, 0.93, 0.56)  # lightgreen
    elif index == 2:
        return Color(1.0, 0.65, 0.0)    # orange
    elif index == 1:
        return Color(1.0, 0.63, 0.48)  # lightcoral
    return Color(1, 1, 1)  # default to white

# 获取所有行数
rows = len(filtered_data)

# 创建格式化请求
for i in range(1, rows + 1):
    color = get_color(filtered_data['Index'].iloc[i - 1])
    cell_format = CellFormat(backgroundColor=color)
    format_cell_range(styled_data_sheet, f"A{i+1}:G{i+1}", cell_format)

print(f"Styled data successfully added to the Google Sheets with sheet name '{styled_data_sheet_name}'.")

In [14]:
#【4】另存为 filtered_data，去除不需要的行和列，添加高亮，更容易阅读分析
# 和上面输到 Google sheet 的内容一样

# 只保留 "HighOrLow" 列不为空字符串的行
filtered_data = original_data[original_data['HighOrLow']!= '']
filtered_data = filtered_data[filtered_data['Index'].isin([1, 2, 3])]

# # 选择需要显示的列
selected_columns = [ "Found_Date", "Found_Close", "Found_OBV" ,"Date","Close", "OBV", "Index" ]

# # 过滤 DataFrame 以只包含选定的列
filtered_data = filtered_data[selected_columns]


# 确保 'Date' 和 'Found_date' 列的类型是 datetime，如果不是需要先转换
filtered_data[['Date', 'Found_Date']] = filtered_data[['Date', 'Found_Date']].apply(pd.to_datetime)
# 将 'Date' 和 'Found_date' 列转换为仅包含日期的格式
filtered_data[['Date', 'Found_Date']] = filtered_data[['Date', 'Found_Date']].apply(lambda x: x.dt.date)
# 将 'Close' 和 'Found_Close' 列转换为数值类型
filtered_data['Close'] = pd.to_numeric(filtered_data['Close'], errors='coerce')
filtered_data['Found_Close'] = pd.to_numeric(filtered_data['Found_Close'], errors='coerce')
# 将 'Close' 和 'Found_Close' 列格式化为小数点后两位并转换为字符串
filtered_data['Close'] = filtered_data['Close'].map('{:.2f}'.format)
filtered_data['Found_Close'] = filtered_data['Found_Close'].map('{:.2f}'.format)
# 确保 'OBV' 和 'Found_OBV' 列转换为数值类型
filtered_data['OBV'] = pd.to_numeric(filtered_data['OBV'], errors='coerce')
filtered_data['Found_OBV'] = pd.to_numeric(filtered_data['Found_OBV'], errors='coerce')
# 缩小 'OBV' 和 'Found_OBV' 列的值，并取整
filtered_data['OBV'] = (filtered_data['OBV'] / scale_factor).round().astype(int)
filtered_data['Found_OBV'] = (filtered_data['Found_OBV'] / scale_factor).round().astype(int)

filtered_data = filtered_data.head(10)

# 高亮显示数据
def highlight_rows(row):
    color = ''
    if row['Index'] == 3:
        color = 'background-color: lightgreen'
    elif row['Index'] == 2:
        color = 'background-color: orange'
    elif row['Index'] == 1:
        color = 'background-color: lightcoral'
    return [color] * len(row)

# 应用高亮函数
styled_data = filtered_data.style.apply(highlight_rows, axis=1)

# 显示高亮后的数据
display(styled_data)

,Found_Date,Found_Close,Found_OBV,Date,Close,OBV,Index
8,2024-04-05,169.58,1620,2024-04-19,165.00,1621,3
9,2024-03-15,172.62,1703,2024-04-12,176.55,1652,1
20,2023-12-15,197.57,1747,2024-01-26,192.42,1756,2
67,2023-02-17,152.55,1381,2023-03-03,151.03,1387,2
68,2023-02-10,151.01,1349,2023-02-24,146.71,1360,3
71,2022-11-18,151.29,1491,2023-02-03,154.50,1382,1
82,2022-10-28,155.74,1458,2022-11-18,151.29,1491,2
116,2022-02-04,172.39,1982,2022-03-25,174.72,1863,1
123,2022-01-14,173.07,1907,2022-02-04,172.39,1982,2
125,2022-01-07,172.17,1865,2022-01-21,162.41,1868,3
